In [11]:
# import necessary packages
import pandas as pd
from transformers import AutoTokenizer, pipeline
import transformers
import torch
from tqdm.auto import tqdm
import time
import os
import json
import sys
import gc
from tqdm import tqdm

In [4]:
# inladen & mergen data
def merge_annotated_data(naam_dataset, n_annotators, n_annotations = 3):
    """ functie die de gesplitste data gesplitst door split_annotated_data weer kan samenvoegen.
    De datasets met de naam naam_dataset_n_annotators.csv worden ingeladen. Vervolgens wordt de oorspronkelijke dataset gereconstrueerd
    uit de ingeladen data. De oorspronkelijke zinnen en het overzicht van de labels wordt gereturned.

    Input:
    naam_dataset: str, de naam van de documenen die de annotators hebben ingevuld
    n_annotators: int, het aantal annotators
    n_annotations: int, het aantal annotations per zin. default 3

    Output:
    merged_data: een DataFrame met daarin de oorspronkelijke data en de n_annotations labels die zijn toegekend.
    """
    # inladen path naar data en creëren lege frames
    dataset_path = "~/Projecten/programma-discriminatie-en-racisme/datasets/Datasets_2/"
    merged_data = pd.DataFrame()

    for i in range(n_annotators):
        # inladen annotated documenten en samenvoegen
        annotated_doc = pd.read_excel(dataset_path + naam_dataset + "_" + str(i) + '.xlsx')
        annotated_doc = annotated_doc.dropna(subset=['Label', 'text', 'Column1'])
        print(len(annotated_doc))
        columns = list(annotated_doc.columns)

        #display(annotated_doc)

        # document opstellen
        if i == 0:
            merged_data = annotated_doc
        else:
            merged_data = pd.concat([merged_data, annotated_doc])
    
    extra_zoekwoorden = pd.read_excel(dataset_path + 'te_verwijderen.xlsx')
    merged_data = pd.concat([merged_data, extra_zoekwoorden])
    
    missende_zoekwoorden = pd.read_excel(dataset_path + 'missing_words.xlsx')
    merged_data = pd.concat([merged_data, missende_zoekwoorden])
    
    extra_migratie = pd.read_excel(dataset_path + 'extra_migratie.xlsx')
    merged_data = pd.concat([merged_data, extra_migratie])
    
    herlabelled = pd.read_excel(dataset_path + 'herlabellen.xlsx')
    merged_data = pd.concat([merged_data, herlabelled])
    
    #display(merged_data)

    aggregated_data = merged_data.groupby(['Column1']).agg({'id': 'last', 'titel': 'last', 'text': 'last', 'zin': 'last', 'word': 'last', 'Label': 'last'})
    
    print(len(merged_data), len(aggregated_data)) 
    
    #aggregated_data = aggregated_data.sort_values(by = ['Column1'])

    return pd.DataFrame(merged_data), pd.DataFrame(aggregated_data)

merged, aggregated = merge_annotated_data('te_labellen', n_annotators = 8, n_annotations = 3)

display(merged)

169
451
423
421
604
772
296
545


/data/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/data/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/data/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/data/anac

5121 4679


,Column1,Column2,Label,aanhangselnummer,alias,api_gewijzigd_op,categorie,citeertitel,content_length,content_type,...,toelichting,type_document,url_document,vergaderjaar,vermijden,verwijderd,volgnummer,voorkeur,word,zin
3,26136,NaN,0.0,NaN,NaN,2024-02-20 06:48:04.095000,migratie,NaN,129657,application/pdf,...,NaN,Hoofddocument,https://gegevensmagazijn.tweedekamer.nl/OData/...,2018-2019,tsunami,0.0,3,NaN,stromen,51
5,18406,NaN,0.0,NaN,NaN,2024-02-23 18:15:17.654000,algemeen,NaN,43492,application/pdf,...,NaN,Hoofddocument,https://gegevensmagazijn.tweedekamer.nl/OData/...,2013-2014,NaN,0.0,4,NaN,macht,2
6,9109,NaN,7.0,NaN,NaN,2024-02-23 07:58:23.236000,talen buiten nl,NaN,1.78453e+06,application/pdf,...,NaN,Hoofddocument,https://gegevensmagazijn.tweedekamer.nl/OData/...,2009-2010,NaN,0.0,2,NaN,armoede,2
7,1887,NaN,0.0,NaN,NaN,2024-02-21 18:01:54.251000,seksualiteit,Wet seksuele misdrijven,287052,application/pdf,...,NaN,Hoofddocument,https://gegevensmagazijn.tweedekamer.nl/OData/...,2022-2023,NaN,0.0,7,NaN,seksueel,13
9,139,NaN,0.0,NaN,NaN,2024-02-21 16:59:39.016000,migratie,Wet vereenvoudiging en modernisering bewijsrecht,195871,application/pdf,...,NaN,Hoofddocument,https://gegevensmagazijn.tweedekamer.nl/OData/...,2021-2022,NaN,0.0,6,NaN,stroom,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,33861,421.0,1.0,NaN,NaN,2024-02-21 05:30:57.870000,gender,NaN,3611171,application/pdf,...,NaN,Hoofddocument,https://gegevensmagazijn.tweedekamer.nl/OData/...,2022-2023,NaN,0.0,7,NaN,kloof,6
57,4478,520.0,1.0,NaN,NaN,2024-02-21 15:39:47.985000,migratie,NaN,133315,application/pdf,...,NaN,Hoofddocument,https://gegevensmagazijn.tweedekamer.nl/OData/...,2016-2017,NaN,0.0,6,NaN,nieuwkomer,2
58,17405,523.0,1.0,NaN,NaN,2024-02-21 12:55:23.639000,algemeen,NaN,34737,application/pdf,...,"vermijd othering, rol slachtoffers",Hoofddocument,https://gegevensmagazijn.tweedekamer.nl/OData/...,2015-2016,NaN,0.0,3,NaN,minderheden,10
59,10024,527.0,1.0,NaN,NaN,2024-02-22 10:30:14.182000,migratie,NaN,5243158,application/pdf,...,NaN,Hoofddocument,https://gegevensmagazijn.tweedekamer.nl/OData/...,2015-2016,NaN,0.0,2,NaN,migranten,1


In [ ]:
# bekijken dataset
print(merged.columns)

pos = len(aggregated[aggregated.Label == 1.0])
neg = len(aggregated[aggregated.Label==0.0])
unclear = len(aggregated[aggregated.Label==4.0])
useless = len(aggregated[aggregated.Label==7.0])

print(f"The dataset has {pos} biased and {neg} unbiased instances for a total of {pos+neg}. There are {unclear} unclear labels and {useless} useless instances.")

In [38]:
# Check if GPU is available and specify GPU ID
device = f"cuda:{gpu_id}" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

model_8b = "unsloth/llama-3-8b-bnb-4bit"

tokenizer_8b = AutoTokenizer.from_pretrained(model_8b)

train_args = {"reprocess_input_data": True, # tokenization gebeurt binnen model
             "fp16":False, # vorm van de input van de data
             "silent": False, # zeker weten dat progress bars worden geprint
             "use_multiprocessing": False, # versnelt training
             "use_multiprocessing_for_evaluation": False, # versnelt training
              "dataloader_num_workers": 0,
             "overwrite_output_dir": True, # niet ieder model opslaan
             "save_total_limit": 2, # slaat meest recente en beste epoch op
             "save_strategy": 'no', # niet tussentijds opslaan
             "load_best_model_at_end": True, # het beste model inladen
             "num_train_epochs": 7, # hoeveel epochs
             "learning_rate": 0.000002, # learning rate
             "weight_decay": 0.15,
             "optimizer": 'AdamW',
             #"loss_type": "custom",
             #"hidden_dropout_prob": 0.5,
             "evaluate_during_training": True, # om validation data in te laten
            "evaluation_strategy":"epoch", # slaat een tussentijds model op om te evalueren
            #"eval_steps": 2000, # hoe vaak het model wordt opgeslagen
            "train_batch_size": 4, # batch size trainingsdata
            "eval_batch_size": 4}

# Specify the device explicitly in the pipeline
pipeline_8b = transformers.pipeline(
    "text-classification",
    model=model_8b,
    args = train_args,
    torch_dtype=torch.float16,
    device=device
)

Using device: cpu


Detected the presence of a `quantization_config` attribute in the model's configuration but you don't have the correct `bitsandbytes` version to support int8 serialization. Please install the latest version of `bitsandbytes` with  `pip install --upgrade bitsandbytes`.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:37                                                                                   │
│                                                                                                  │
│ /data/milenadeswart/.local/lib/python3.7/site-packages/transformers/pipelines/__init__.py:795 in │
│ pipeline                                                                                         │
│                                                                                                  │
│   792 │   │   framework=framework,                                                               │
│   793 │   │   task=task,                                                                         │
│   794 │   │   **hub_kwargs,                                                                      │
│ ❱ 795 │   │   **model_kwargs,                                                                    │
│   796 │   )                                                                                      │
│   797 │                                                                                          │
│   798 │   model_config = model.config                                                            │
│                                                                                                  │
│ /data/milenadeswart/.local/lib/python3.7/site-packages/transformers/pipelines/base.py:269 in     │
│ infer_framework_load_model                                                                       │
│                                                                                                  │
│    266 │   │   │   │   )                                                                         │
│    267 │   │   │                                                                                 │
│    268 │   │   │   try:                                                                          │
│ ❱  269 │   │   │   │   model = model_class.from_pretrained(model, **kwargs)                      │
│    270 │   │   │   │   if hasattr(model, "eval"):                                                │
│    271 │   │   │   │   │   model = model.eval()                                                  │
│    272 │   │   │   │   # Stop loading on the first successful load.                              │
│                                                                                                  │
│ /data/milenadeswart/.local/lib/python3.7/site-packages/transformers/models/auto/auto_factory.py: │
│ 485 in from_pretrained                                                                           │
│                                                                                                  │
│   482 │   │   elif type(config) in cls._model_mapping.keys():                                    │
│   483 │   │   │   model_class = _get_model_class(config, cls._model_mapping)                     │
│   484 │   │   │   return model_class.from_pretrained(                                            │
│ ❱ 485 │   │   │   │   pretrained_model_name_or_path, *model_args, config=config, **hub_kwargs,   │
│   486 │   │   │   )                                                                              │
│   487 │   │   raise ValueError(                                                                  │
│   488 │   │   │   f"Unrecognized configuration class {config.__class__} for this kind of AutoM   │
│                                                                                                  │
│ /data/milenadeswart/.local/lib/python3.7/site-packages/transformers/modeling_utils.py:2604 in    │
│ from_pretrained                                                                                  │
│                                                                                                  │
│   2601 │   │   if from_pt:                                 

In [28]:
model, tokenizer = FastLanguageModel.from_pretrained(
     model_name = model_8b,
     args = train_args,
     max_seq_length = max_seq_length,
     dtype = dtype,
     load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'FastLanguageModel' is not defined